In [13]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import balloon

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

import warnings
warnings.filterwarnings("ignore")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline

In [10]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join("C:\\Users\\YASIR\\Desktop\\New folder (2)\\Mask_RCNN-master\\logs", "mask_rcnn_wall_0118.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "samples\balloon\dataset\train")

In [14]:
config = balloon.BalloonConfig()
class InferenceConfig(config.__class__):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [16]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir='mask_rcnn_wall_0118.hy', config=config)

# Load weights trained on MS-COCO
model.load_weights('mask_rcnn_wall_0118.h5', by_name=True)

OSError: Unable to open file (truncated file: eof = 1376256, sblock->base_addr = 0, stored_eof = 257557808)

In [12]:
from django.shortcuts import render,HttpResponse
import imutils
from imutils import contours
from scipy.spatial import distance as dist
from imutils import perspective
import cv2
import numpy as np
from skimage import io, color
from itertools import chain
from keras import backend as K

def find_in_2d_array(arr, value):
    return value in chain.from_iterable(arr)
def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

def sizefinder(VER_HOR,count):
    pixelsPerMetric=None
    gray = cv2.GaussianBlur(VER_HOR, (7, 7), 0)
    edged = cv2.Canny(gray, 50, 100)
    edged = cv2.dilate(edged, None, iterations=1)
    bo = True
    cnts = cv2.findContours(edged, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    if(cnts != []):
        (cnts, _) = contours.sort_contours(cnts)
        points_HOR = {}
        points_VER = {}
        for c in cnts:
            if cv2.contourArea(c) < 100:
                continue
            orig = VER_HOR.copy()
            box = cv2.minAreaRect(c)
            box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
            box = np.array(box, dtype="int")
            box = perspective.order_points(box)
            cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)
            for (x, y) in box:
                cv2.circle(orig, (int(x), int(y)), 2, (0, 0, 255), -1)
            (tl, tr, br, bl) = box
            (tltrX, tltrY) = midpoint(tl, tr)
            (blbrX, blbrY) = midpoint(bl, br)
            (tlblX, tlblY) = midpoint(tl, bl)
            (trbrX, trbrY) = midpoint(tr, br)
            points_VER.update({str(count):{'X1':tltrX,'Y1':tltrY,'X2':blbrX,'Y2':blbrY}})
            points_HOR.update({str(count):{'X1':tlblX,'Y1':tlblY,'X2':trbrX,'Y2':trbrY}})

            count+=1
        return points_HOR,points_VER,count,bo
    else:
        return {},{},count,bo

def H_W(IMAGE_NAME):
    im_gray = cv2.imread(IMAGE_NAME,cv2.IMREAD_GRAYSCALE)
    h,w = im_gray.shape[:2]
    return w,h
def horizontal_seperation(bw,basin=25):
    import cv2
    horizontal = np.copy(bw)
    cols = horizontal.shape[1]
    horizontal_size = cols / basin
    horizontal_size=int(horizontal_size)
    horizontalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (horizontal_size, 1))
    horizontal = cv2.erode(horizontal, horizontalStructure)
    horizontal = cv2.dilate(horizontal, horizontalStructure) 
    return horizontal


def vertical_seperation(bw,basin=25):
    import cv2
    vertical = np.copy(bw)
    rows = vertical.shape[0]
    verticalsize = rows / basin
    verticalsize = int(verticalsize)
    verticalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (1, verticalsize))
    vertical = cv2.erode(vertical, verticalStructure)
    vertical = cv2.dilate(vertical, verticalStructure)
    return vertical

def get_ax(rows=1, cols=1, size=16):
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

def home():
    import json
    import os
    import sys
    import random
    import math
    import re
    import time
    import numpy as np
    import tensorflow as tf
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    ROOT_DIR = os.path.abspath(r"C:\Users\yasir\Desktop\sites\main\Mask_RCNN-master")
    sys.path.append(ROOT_DIR) 
    from mrcnn import utils
    from mrcnn import visualize
    from mrcnn.visualize import display_images
    import mrcnn.model as modellib
    from mrcnn.model import log
    from samples.balloon import balloon 
    np.set_printoptions(threshold=sys.maxsize)
    sess= tf.Session()
    directory = r"C:\Users\yasir\Desktop\sites\main\Mask_RCNN-master\samples\balloon\dataset\train\Ib_GY0301_sommaire.jpg"
    


    MODEL_DIR = os.path.join(ROOT_DIR, "logs/mask_rcnn_wall_0118.h5")
    BALLON_WEIGHTS_PATH = r"C:\Users\yasir\Desktop\sites\main\Mask_RCNN-master\logs\mask_rcnn_wall_0118.h5"  # TODO: update this path
    config = balloon.BalloonConfig()
    BALLOON_DIR = os.path.join(ROOT_DIR, "samples/balloon/dataset/")
    class InferenceConfig(config.__class__):
        GPU_COUNT = 1
        IMAGES_PER_GPU = 1
    config = InferenceConfig()
    config.display()
    DEVICE = "/gpu:0" 
    TEST_MODE = "inference"
    K.clear_session()
    with tf.device(DEVICE):
        model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                                  config=config)
    model.load_weights(r"C:\Users\yasir\Desktop\sites\main\Mask_RCNN-master\logs\mask_rcnn_wall_0118.h5", by_name=True)
    class_names = ['BG', 'wall']
    from skimage import filters 
    from skimage import data 
    import skimage
    image = skimage.io.imread(directory)
    results = model.detect([image], verbose=1)
    K.clear_session()
    r = results[0]
    mask = r['masks']
    mask = mask.astype(int)
    import skimage
    temp = skimage.io.imread(directory)
    w,h,e = temp.shape[:3]
    points_VX = {}
    points_VY = {}
    points_HX = {}
    points_HY = {}
    count=0
    count1=0
    for i in range(mask.shape[2]):
        temp = np.zeros((w,h,e),dtype=np.uint8)
        temp = np.where(temp==0,255,temp)
        for j in range(temp.shape[2]):
            temp[:,:,j] = temp[:,:,j] * mask[:,:,i]
        a1,b1,cou,bo = sizefinder(vertical_seperation(temp),count)
        count = cou
        if(bo):
            points_VX.update(a1)
            points_VY.update(b1)
        c1,d1,cou1,bo = sizefinder(horizontal_seperation(temp),count1)
        count1 = cou1
        if(bo):
            points_HX.update(c1)
            points_HY.update(d1)
    w,h = H_W(directory)
    points_HX = json.dumps(points_HX)
    points_VY = json.dumps(points_VY)
home()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         